In [1]:
import pandas as pd
import numpy as np
from EEGFeatureExtraction import generate_feature_vectors_from_samples

In [2]:
import pickle

with open('congruent_features_bas_150.pickle', 'rb') as output:
    data_array = pickle.load(output)


with open('congruent_labels_bas_150.pickle', 'rb') as output:
      label_array = pickle.load(output)


with open('congruent_groups_bas_150.pickle', 'rb') as output:
      group_array = pickle.load(output)   

In [3]:
data =  data_array[0,:,:]
data = data.T
results, names = generate_feature_vectors_from_samples(data, 150, 1, remove_redundant = False)
# data = pd.DataFrame(data=results, columns=names)

c:\Users\erangad\AppData\Local\anaconda3\envs\env\lib\site-packages\scipy\linalg\_matfuncs_inv_ssq.py:836: LogmNearlySingularWarning: The logm input matrix may be nearly singular.
  warnings.warn(near_singularity_msg, LogmNearlySingularWarning)


In [4]:
len(names)

468

In [6]:
FINAL_MATRIX = None
for i in range(0,data_array.shape[0]):
    datai =  data_array[i,:,:]
    datai = datai.T   

    vectors, header = generate_feature_vectors_from_samples(datai, 150, 1.0, remove_redundant = False)
    # print ('resulting vector shape for the file', vectors.shape)

    if (FINAL_MATRIX is None):
      FINAL_MATRIX = vectors
    else:
       FINAL_MATRIX = np.vstack([ FINAL_MATRIX, vectors ])  
      
print(FINAL_MATRIX.shape)	
matrix  = pd.DataFrame(FINAL_MATRIX,  columns = header, index = None)    
matrix['label']  = label_array
matrix['group']  = group_array

    

c:\Users\erangad\AppData\Local\anaconda3\envs\env\lib\site-packages\scipy\linalg\_matfuncs_inv_ssq.py:836: LogmNearlySingularWarning: The logm input matrix may be nearly singular.
  warnings.warn(near_singularity_msg, LogmNearlySingularWarning)
c:\Users\erangad\AppData\Local\anaconda3\envs\env\lib\site-packages\scipy\linalg\_matfuncs_inv_ssq.py:827: LogmExactlySingularWarning: The logm input matrix is exactly singular.
  warnings.warn(exact_singularity_msg, LogmExactlySingularWarning)


logm result may be inaccurate, approximate err = 6.439072526055541e-10
logm result may be inaccurate, approximate err = 5.478368907365862e-10
logm result may be inaccurate, approximate err = 1.426970539744213e-09
logm result may be inaccurate, approximate err = 1.108619612481419e-09
logm result may be inaccurate, approximate err = 8.692833798297625e-10
logm result may be inaccurate, approximate err = 5.254894318449786e-10
logm result may be inaccurate, approximate err = 5.869570316010781e-10
logm result may be inaccurate, approximate err = 9.065957781745395e-10
logm result may be inaccurate, approximate err = 6.803153774021666e-10
logm result may be inaccurate, approximate err = 5.776259479972392e-10
logm result may be inaccurate, approximate err = 6.859078866697539e-10
logm result may be inaccurate, approximate err = 6.030051263179324e-10
logm result may be inaccurate, approximate err = 7.367265623743147e-10
logm result may be inaccurate, approximate err = 8.549475659969587e-10
logm r

In [8]:
matrix.to_pickle("./matrix_congruent_bas_150.pkl")

In [9]:
import pandas as pd
from sklearn import feature_selection as fs

data = pd.read_pickle("./matrix_congruent_bas_150.pkl")  

x_train = data.drop(['label','group'], axis=1)
y_train = data['label']
group = data['group']

In [11]:
#     # Feature selection - k = number of features to select
# from sklearn.model_selection import LeaveOneGroupOut
# from sklearn.feature_selection import RFECV
# # gkf=GroupKFold(n_splits=5)
# logo = LeaveOneGroupOut()

# select_beast_selector = fs.SelectKBest(fs.f_classif, k=256)

# selector = RFECV(select_beast_selector, step=1, cv=logo)
# selector = selector.fit_transform(x_train, y_train, groups=group_array)

TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator SelectKBest(k=256) does not.

In [10]:
    # Feature selection - k = number of features to select
    feature_selector = fs.SelectKBest(fs.f_classif, k=256)
    feature_selector.fit_transform(x_train, y_train)
    mask = feature_selector.get_support()

    # List of all feature names
    feature_names = list(data.columns.values)

    # List of selected feature names
    new_feature_names = []
    for feature_is_selected, feature in zip(mask, feature_names):
        if feature_is_selected:
            new_feature_names.append(feature)


In [12]:
with open('selected_features_bas_150.pickle', 'wb') as output:
    pickle.dump(new_feature_names, output)

In [13]:
feature_names_plus_label_group = new_feature_names.copy()
feature_names_plus_label_group.append("label")
feature_names_plus_label_group.append("group")
selected_data = data[feature_names_plus_label_group].copy()

x = selected_data.drop(['label','group'], axis=1)
y = selected_data['label']
group = selected_data['group']

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold,GridSearchCV,cross_val_score,cross_validate 
from sklearn.model_selection import LeaveOneGroupOut

logo = LeaveOneGroupOut()

In [15]:
clf=LogisticRegression()
logo = LeaveOneGroupOut()
param_grid = {'classifier__C': [0.001, 0.005, 0.01,0.05,0.1,0.5, 1,2,3,4,5,8, 10,12,15]}
pipe=Pipeline([('scaler',StandardScaler()),('classifier',clf)])
gscv=GridSearchCV(pipe,param_grid,cv=logo,n_jobs=16)
gscv.fit(x,y,groups=group)


GridSearchCV(cv=LeaveOneGroupOut(),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier', LogisticRegression())]),
             n_jobs=16,
             param_grid={'classifier__C': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5,
                                           1, 2, 3, 4, 5, 8, 10, 12, 15]})

In [16]:
gscv.best_score_ , gscv.best_params_

(0.5361665829346122, {'classifier__C': 0.001})

In [17]:
from sklearn.svm import SVC
param_grid = {'classifier__C': [0.1, 1, 10, 100, 1000], 
              'classifier__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'classifier__kernel': ['rbf']} 

pipe = Pipeline([ ('scaler', StandardScaler()),
                 ('classifier', SVC())])
logo = LeaveOneGroupOut()
gscv=GridSearchCV(pipe,param_grid,cv=logo,n_jobs=16)
gscv.fit(x,y,groups=group)

GridSearchCV(cv=LeaveOneGroupOut(),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier', SVC())]),
             n_jobs=16,
             param_grid={'classifier__C': [0.1, 1, 10, 100, 1000],
                         'classifier__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'classifier__kernel': ['rbf']})

In [18]:
gscv.best_score_ , gscv.best_params_

(0.5390755113257021,
 {'classifier__C': 10,
  'classifier__gamma': 0.0001,
  'classifier__kernel': 'rbf'})

In [19]:
from sklearn.ensemble import RandomForestClassifier

param_grid = { 
    'classifier__n_estimators': [200, 500],
    'classifier__max_features': ['auto', 'sqrt', 'log2'],
    'classifier__max_depth' : [4,5,6,7,8],
    'classifier__criterion' :['gini', 'entropy']
}

clf = RandomForestClassifier(random_state=42)

pipe = Pipeline([ ('scaler', StandardScaler()),
                 ('classifier', clf)])
logo = LeaveOneGroupOut()
gscv=GridSearchCV(pipe,param_grid,cv=logo,n_jobs=16)
gscv.fit(x,y,groups=group)

c:\Users\erangad\AppData\Local\anaconda3\envs\env\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
400 fits failed out of a total of 1200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
219 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\erangad\AppData\Local\anaconda3\envs\env\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\erangad\AppData\Local\anaconda3\envs\env\lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\erangad\AppData\Local\anaconda3\envs\env\lib\site-packages\sklearn\pipeli

GridSearchCV(cv=LeaveOneGroupOut(),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=16,
             param_grid={'classifier__criterion': ['gini', 'entropy'],
                         'classifier__max_depth': [4, 5, 6, 7, 8],
                         'classifier__max_features': ['auto', 'sqrt', 'log2'],
                         'classifier__n_estimators': [200, 500]})

In [20]:
gscv.best_score_ , gscv.best_params_

(0.5351505155392677,
 {'classifier__criterion': 'entropy',
  'classifier__max_depth': 8,
  'classifier__max_features': 'sqrt',
  'classifier__n_estimators': 500})

In [21]:
import xgboost as xgb

param_grid = {
        'classifier__min_child_weight': [1, 5, 10],
        'classifier__gamma': [0.5, 1, 1.5, 2, 5],
        'classifier__subsample': [0.6, 0.8, 1.0],
        'classifier__colsample_bytree': [0.6, 0.8, 1.0],
        'classifier__max_depth': [3, 4, 5]
        }

clf = xgb.XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

pipe = Pipeline([ ('scaler', StandardScaler()),
                 ('classifier', clf)])
logo = LeaveOneGroupOut()
gscv=GridSearchCV(pipe,param_grid,cv=logo,n_jobs=16)
gscv.fit(x,y,groups=group)

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
gscv.best_score_ , gscv.best_params_

In [22]:
from tensorflow.keras.layers import Conv1D,BatchNormalization,LeakyReLU,MaxPool1D,\
GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from keras.layers import Conv2D, MaxPooling2D # new!

def cnnmodel():
    clear_session()
    model=Sequential()
    
    model.add(Conv2D(filters=32,kernel_size=3,strides=1,input_shape=(16,16,1)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    # model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=64,kernel_size=3,strides=1))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(0.25))

    model.add(Flatten())

    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.25))

    model.add(Dense(1, activation='sigmoid'))
    
    model.compile('adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

model=cnnmodel()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 14, 14, 32)       128       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 14, 14, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 12, 12, 64)       256       
 hNormalization)                                                 
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 12, 12, 64)        0

In [23]:
data_array = x.to_numpy()
label_array = y.to_numpy()
group_array = group.to_numpy()

accuracy=[]
for train_index, val_index in logo.split(data_array, label_array, groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler()
    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    train_features = train_features.reshape((-1,16,16 ))
    val_features = val_features.reshape((-1,16,16 ))

    print(train_features.shape)
    model=cnnmodel()
    model.fit(train_features,train_labels,epochs=50,batch_size=64,validation_data=(val_features,val_labels))
    accuracy.append(model.evaluate(val_features,val_labels)[1])

(1488, 16, 16)
Epoch 1/50
24/24 [==============================] - 2s 41ms/step - loss: 1.6537 - accuracy: 0.4953 - val_loss: 0.6911 - val_accuracy: 0.5579
Epoch 2/50
24/24 [==============================] - 1s 29ms/step - loss: 0.6955 - accuracy: 0.5336 - val_loss: 0.6893 - val_accuracy: 0.5684
Epoch 3/50
24/24 [==============================] - 1s 28ms/step - loss: 0.6819 - accuracy: 0.5524 - val_loss: 0.6911 - val_accuracy: 0.5158
Epoch 4/50
24/24 [==============================] - 1s 28ms/step - loss: 0.6730 - accuracy: 0.5706 - val_loss: 0.6932 - val_accuracy: 0.5579
Epoch 5/50
24/24 [==============================] - 1s 28ms/step - loss: 0.6769 - accuracy: 0.5638 - val_loss: 0.6961 - val_accuracy: 0.4947
Epoch 6/50
24/24 [==============================] - 1s 28ms/step - loss: 0.6681 - accuracy: 0.5806 - val_loss: 0.6982 - val_accuracy: 0.5158
Epoch 7/50
24/24 [==============================] - 1s 28ms/step - loss: 0.6618 - accuracy: 0.6069 - val_loss: 0.6911 - val_accuracy: 0.568

In [24]:
np.mean(accuracy)

0.5185015708208084